# Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [4]:
df = pd.read_csv('data_with_geo_points.csv')

In [5]:
df

,Unnamed: 0.1,Unnamed: 0,name,title,ID,permit_num,permit_type,review_type,app_start_date,issue_date,...,reported_cost,ward,full_address,app_year_start,geo_address,loc,point,lat,lon,altitude
0,0,0,DEWALT MARK D,ARCHITECT,2918165.0,100747649,PERMIT - NEW CONSTRUCTION,TRADITIONAL DEVELOPER SERVICES,2018-02-16,03/13/2018,...,8000000.0,25.0,1000 W MONROE ST,2018.0,"1000 W MONROE ST Chicago, IL","1000 W Monroe St, Chicago, IL 60607, USA","(41.8805916, -87.6521977, 0.0)",41.880592,-87.652198,0.0
1,1,1,HARTSHORNE RAY,ARCHITECT,2944797.0,100764129,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2018-07-31,08/08/2018,...,12000000.0,2.0,810 N CLARK ST,2018.0,"810 N CLARK ST Chicago, IL","810 N Clark St, Chicago, IL 60610, USA","(41.8970694, -87.63155909999999, 0.0)",41.897069,-87.631559,0.0
2,2,2,WINEBERG CLARA INES,ARCHITECT,3184534.0,100921065,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2021-07-26,09/27/2021,...,675000.0,27.0,166 N ABERDEEN ST,2021.0,"166 N ABERDEEN ST Chicago, IL","166 N Aberdeen St, Chicago, IL 60607, USA","(41.8850049, -87.6547992, 0.0)",41.885005,-87.654799,0.0
3,3,3,NUTTELMAN JASON P,ARCHITECT,2878062.0,100724116,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2017-09-28,11/15/2017,...,17442000.0,20.0,5504 S MICHIGAN AVE,2017.0,"5504 S MICHIGAN AVE Chicago, IL","5504 S Michigan Ave, Chicago, IL 60615, USA","(41.7941921, -87.6228129, 0.0)",41.794192,-87.622813,0.0
4,4,4,"PAISLEY, KIRK",ARCHITECT,2987209.0,100790354,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2019-01-22,03/13/2019,...,10800000.0,14.0,4835 S CICERO AVE,2019.0,"4835 S CICERO AVE Chicago, IL","4835 S Cicero Ave, Chicago, IL 60632, USA","(41.804736, -87.742685, 0.0)",41.804736,-87.742685,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,676,676,WATERS ROBERT,ARCHITECT,3000.0,100903482,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2021-01-20,01/29/2021,...,206000.0,10.0,3408 E 118TH ST,2021.0,"3408 E 118TH ST Chicago, IL","3408 E 118th St, Chicago, IL 60617, USA","(41.6816741, -87.541731, 0.0)",41.681674,-87.541731,0.0
655,677,677,GIL RUBEN,ARCHITECT,777.6,100907299,PERMIT - NEW CONSTRUCTION,TRADITIONAL DEVELOPER SERVICES,2021-01-28,01/28/2021,...,20000.0,5.0,5602 S MARYLAND AVE,2021.0,"5602 S MARYLAND AVE Chicago, IL","5602 S Maryland Ave, Chicago, IL 60637, USA","(41.7930775, -87.6052314, 0.0)",41.793078,-87.605231,0.0
656,678,678,STEIN JONATHAN,ARCHITECT,13407.9,100908460,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2021-02-16,02/23/2021,...,200000.0,27.0,311 N GREEN ST,2021.0,"311 N GREEN ST Chicago, IL","311 N Green St, Chicago, IL 60607, USA","(41.8872037, -87.6485191, 0.0)",41.887204,-87.648519,0.0
657,679,679,GELDIS GEORGE,ARCHITECT,0.0,100910538,PERMIT - NEW CONSTRUCTION,TRADITIONAL DEVELOPER SERVICES,2021-02-26,02/26/2021,...,3500000.0,37.0,4443 W CHICAGO AVE,2021.0,"4443 W CHICAGO AVE Chicago, IL","4443 W Chicago Ave, Chicago, IL 60651, USA","(41.8951612, -87.7370103, 0.0)",41.895161,-87.737010,0.0


In [7]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    architect = df['name'].iloc[i]
    work_desc = df['work_desc'].iloc[i]
    issue_date = df['issue_date'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Architect: <strong>{}'''.format(architect) + '''</strong><br>
    Description: <strong>{}'''.format(work_desc) + '''</strong><br>
    Issue Date: <strong>{}'''.format(issue_date) + '''</strong><br>
    </html>
    '''
    return html

## No popup or Tooltip - Pins Only

In [20]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

for i in range(0,len(df)):
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]]).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

m

## Popup attempt

In [13]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

m

## Tooltip attempt

In [12]:
import folium
from folium.plugins import MarkerCluster
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

marker_cluster = MarkerCluster().add_to(m)

for i, r in df.iterrows():
    location = (r['lat'], r['lon'])
    info = (r['full_address'],r['name'],r['work_desc'],r['issue_date'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Address: {bold_start}{info[0]}{bold_end}{new_line}\
    Architect: {bold_start}{info[1]}{bold_end}{new_line}\
    Description: {bold_start}{info[2]}{bold_end}{new_line}\
    Issue Date: {bold_start}{info[3]}{bold_end}{new_line}'
    folium.Marker(location=location,tooltip=text).add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

m